In [ ]:
!wget https://www.dropbox.com/s/zp8m6v69kvfk2os/images-2.zip?dl=1

In [ ]:
!wget https://www.dropbox.com/s/wq4f8m959yloai3/video_frames_long.zip?dl=1

In [ ]:
!conda install pytorch==1.2.0 torchvision==0.4.0 cudatoolkit=10.0 -c pytorch

In [ ]:
#Выше ссылки для скачивания архивов с раскадровкой и изображениями.

In [ ]:
import cv2
import os
import numpy as np
import shutil
import torch
import zipfile
import cv2
import os
import numpy as np
import pandas as pd
import datetime as dt
dataset = pd.read_csv('dataset.csv')

In [ ]:
dataset['Time_cur'] = pd.to_datetime(dataset['published_at'])
dataset['published_hour_cur'] = dataset['Time_cur'].dt.hour
dataset['Weekday_cur'] = dataset['Time_cur'].dt.weekday
diff = dt.datetime.now() - dataset['Time_cur']
dataset['Published_cur_time'] = (dt.datetime.now() - dataset['Time_cur']).dt.total_seconds()
dataset  = dataset.drop(['Time_cur', 'published_at'], axis=1)

In [ ]:
dataset = dataset.sort_values(by = 'Published_cur_time',   ascending=False)

In [ ]:
dataset['channel_founded'] = pd.to_datetime(dataset['channel_founded'])
diff = dt.datetime.now() - dataset['channel_founded']
dataset['channel_founded'] = diff.dt.total_seconds()

In [ ]:
dataset['popular'] = 0.0

In [ ]:
dataset['popular'][dataset.view_count_new.gt(dataset.view_count_new.quantile(0.8))] = 1.0

In [ ]:
# Кодировка категориальных признаков

In [ ]:
from sklearn.preprocessing import OneHotEncoder
dataset = pd.get_dummies(dataset, columns=['published_hour_cur', 'Weekday_cur'], prefix=["published_hour", "Weekday_cur"] )

In [ ]:
#Нормирование вещественных признаков

In [ ]:
from sklearn.preprocessing import StandardScaler
dataset_norm = dataset[['channel_founded', 'subcriber_count', 'view_count', 'like_count', 'dislike_count', 'favorite_count', 'comment_count',  'Published_cur_time' ]]
dataset_norm  = dataset_norm.values
scaler = StandardScaler()
dataset_norm = scaler.fit_transform(dataset_norm)
dataset[['channel_founded', 'subcriber_count', 'view_count', 'like_count', 'dislike_count', 'favorite_count', 'comment_count',  'Published_cur_time' ]] = dataset_norm 







In [ ]:
dataset = dataset[~dataset.isin([np.nan, np.inf, -np.inf]).any(1)]
dataset = dataset.reset_index()

# Обработка видео

In [ ]:
import zipfile
with zipfile.ZipFile("videos_11k.zip", 'r') as zip_ref:
    zip_ref.extractall('videos11_k')

In [ ]:
not_pres = []
def video_frames(video_folder, new_folder, snap_full = True):
    videos = os.listdir(path = video_folder) 
    os.mkdir(new_folder)
    for elem in videos:
        cap = cv2.VideoCapture(video_folder+elem)
        fps = int(cap.get(5))
        lp = np.linspace(33, fps * 10 , num = 20)
        lp = lp.astype('int')
        frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        frames = int(frames)
        if snap_full == True:  
            lp = np.linspace(33, frames-4, num = 20)
            lp = lp.astype('int')
        os.mkdir(new_folder + elem.split('.')[0])
        for ind in lp:
            cap.set(cv2.CAP_PROP_POS_FRAMES, ind)
            ret, frame = cap.read()
            if ret == False:
                print('hello')
                print(elem)
                not_pres.append(elem)
            else:
                cv2.imwrite(new_folder+elem.split('.')[0] + '/'+ str(ind) + '.png', frame)

In [ ]:
video_frames("videos/videos/", 'video_frames_normal/', snap_full = True)

In [ ]:
not_pres = list(set(not_pres))
from pathlib import Path
import shutil
for elem in not_pres:
    adr = elem.split('.')[0]
    dirpath = Path('video_frames_normal', adr)
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)

In [ ]:
import pandas as pd
import datetime as dt
dataset['video_names'] = ''
i = 0
for elem in dataset['video']:
    dataset['video_names'][i] = elem.split('.')[0]
    i+=1

In [ ]:
elems = os.listdir('video_frames_long/video_frames_long')
dataset = dataset[dataset['video_names'].isin(elems)]

In [ ]:
dataset = dataset.reset_index(drop = True)

# Текст

In [ ]:
!pip install spacy

In [ ]:
!pip install spacy
!python -m spacy download en

In [ ]:
# Применение spacy для токенизирования текста.

In [ ]:
import spacy
import re
import string
tok = spacy.load('en')
#Применение regex для удаления  пунктуации
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    text = no_specials_string = re.sub('[!#?,.:";]', ' ', text)
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", text.lower())
    tmp = [token.text for token in tok.tokenizer(nopunct)]
    return [x for x in tmp if not x.isspace()]

In [ ]:
for i in range(len(dataset)):
    dataset['video_title'][i] = tokenize(dataset['video_title'][i])

In [ ]:
vocab2id = {"":0, "UNK":1}
words = ["", "UNK"]
#Составление словаря и создание уникального идентификатора для каждого слова.
from collections import Counter
counts_words = Counter()
for row in dataset['video_title']:
    counts_words.update(row)
for word in counts_words:
    vocab2id[word] = len(words)
    words.append(word)
encoded_texts = []
dataset['num_words'] = 0
for i in range(len(dataset)):
    dataset['num_words'][i] = len(dataset['video_title'][i])

In [ ]:
#Удаление элементов с пустым оглавлением
dataset = dataset[dataset['num_words'] != 0]

In [ ]:
dataset = dataset.reset_index(drop = True)

In [ ]:
#Замена слов в заголовках на уникальные идентификаторы
max_len_title = max(dataset['num_words'])

In [ ]:
#Замена слов в заголовках на уникальные идентификаторы
for elem in dataset['video_title']:
    enc = np.zeros(max_len_title)
    smth = np.array([vocab2id.get(word, vocab2id["UNK"]) for word in elem])
    enc[:len(smth)] = smth
    encoded_texts.append(enc)

In [ ]:
encoded_texts = np.array(encoded_texts,  dtype =  int)

In [ ]:
tmp = pd.Series( (v for v in encoded_texts) )

In [ ]:
dataset['Encoded'] = tmp

In [ ]:
!pip install wget

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

In [ ]:
#Скачивание предобученных эмбеддингов GLOVE
word_vectors = {}
with open("glove/glove.twitter.27B.100d.txt") as f:
    for line in f:
        arr = line.split()
        word_vectors[arr[0]] = np.array([float(x) for x in arr[1:]])

In [ ]:
# Cоставление матрицы эмбеддингов для словаря 
vocab_size = len(counts_words) + 2
vocab_to_idx = {}
vocab = ["", "UNK"]
W = np.zeros((vocab_size, 100), dtype="float32")
W[0] = np.zeros(100, dtype='float32')
W[1] = np.random.uniform(-0.25, 0.25, 100)
vocab_to_idx["UNK"] = 1
i = 2
for word in counts_words:
    if word in word_vectors:
        W[i] = word_vectors[word]
    else:
        W[i] = np.random.uniform(-0.25,0.25, 100)
    vocab_to_idx[word] = i
    vocab.append(word)
    i += 1   

# Загрузка данных

In [ ]:
pip install --upgrade torch torchvision

In [ ]:
#  Создание класса data.dataset, который необходим для загрузки данных в модель MLP. 
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
class Video_fin_dataset(data.Dataset):
    def __init__(self, numerical, cat, data_path_frame, frame_folders, data_path_image, images, text, y,  transform_frames=None, transform_image = None):
        "Initialization"
        self.data_path_frame = data_path_frame
        self.data_path_image = data_path_image
        self.frame_folders = frame_folders
        self.images = images
        self.text = text
        self.transform_frames = transform_frames
        self.transform_image = transform_image
        self.numerical = numerical
        self.cat = cat
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def read_images_folder(self, folder):
        frames_vid = os.listdir(os.path.join(self.data_path_frame, folder))
        def myFunc(e):
            return int(e.split('.')[0])
        out = []
        frames_vid.sort(reverse=False, key=myFunc)
        for pic in frames_vid:
            image = Image.open(os.path.join(self.data_path_frame, folder, pic))
            if self.transform_frames:
                image = self.transform_frames(image)
            out.append(image)
        out = torch.stack(out, dim = 0)
        return out
    
    def __getitem__(self, index):
        encoded_text = torch.from_numpy(self.text[index][0])
        length_text = self.text[index][1]
        num = torch.from_numpy(self.numerical[index])
        cat = torch.from_numpy(self.cat[index])
        folder_frames = self.frame_folders[index]
        out = self.read_images_folder(folder_frames)
        image = self.images[index]
        image = Image.open(os.path.join(self.data_path_image, image))
        image = self.transform_image(image)
        y = list(self.y)[index]
        return num, cat, (encoded_text, length_text), out, image, y

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=12)

In [ ]:
train.columns

In [ ]:
train_numerical = train[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']].values
train_categorical = train[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']].values
train_text = train[['Encoded', 'num_words']].values
train_img = train['review'].values
train_vids = train['video_names'].values
train_target = train[['popular']].values

In [ ]:
val_numerical = val[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']].values
val_categorical = val[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']].values
val_text = val[['Encoded', 'num_words']].values
val_img = val['review'].values
val_vids = val['video_names'].values
val_target = val[['popular']].values

In [ ]:
data_path_frame = 'video_frames_long/video_frames_long/'
data_path_image = 'images-2/images-2/'

In [ ]:
from torchvision import transforms
transform_train_vid = transforms.Compose([transforms.Resize([340, 240]),
                                transforms.RandomCrop([169, 169]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
transform_test_vid = transforms.Compose([transforms.Resize([169, 169]),  transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])])

In [ ]:
transform_train_img = transforms.Compose([transforms.Resize([200, 200]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
transform_test_img = transforms.Compose([transforms.Resize([256, 256]),  transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])])

In [ ]:
dataset_train = Video_fin_dataset(train_numerical, train_categorical, data_path_frame, train_vids, data_path_image, train_img, train_text, train_target, transform_train_vid, transform_train_img)

In [ ]:
dataset_val = Video_fin_dataset(val_numerical, val_categorical, data_path_frame, val_vids, data_path_image, val_img, val_text, val_target, transform_test_vid, transform_test_img)

In [ ]:
train_dl = data.DataLoader(dataset_train, batch_size = 128)
val_dl = data.DataLoader(dataset_val, batch_size = 32)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence
class BiLSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers,  glove_weights):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False
        self.lstm = nn.LSTM(input_size = embedding_dim,
                            hidden_size = 128,
                            bidirectional = True,
                            num_layers = num_layers,
                            dropout = 0.4,
                            batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.func1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(128, 32)
        

    def forward(self, x, l):
        x = self.embeddings(x)
        x_pack = pack_padded_sequence(x, l, batch_first=True, enforce_sorted=False)
        out_st, (ht, ct) = self.lstm(x_pack)
        ht = torch.cat((ht[-2,:,:], ht[-1,:,:]),  dim = 1)
        out = self.fc1(ht)
        out = self.func1(out)
        out = self.fc2(out)
        return out

In [ ]:
class image(torch.nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet152(pretrained=True)
        in_feats = resnet.fc.in_features
        print(in_feats)
        base_mod = list(resnet.children())[:-1]
        self.base_mod = nn.Sequential(*base_mod)
        for child in self.base_mod.children():
            for param in child.parameters():
                param.requires_grad = False
        self.out1 = nn.Linear(in_feats, 256)
        self.func1 = nn.LeakyReLU(0.1)
        self.out2 = nn.Linear(256, 32)

    def forward(self, x):
        x  = self.base_mod(x)
        print(x.shape)
        x =  x.view(-1, 2048)
        x = self.out1(x)
        x = self.func1(x)
        return self.out2(x)

In [ ]:
class video_analysis(nn.Module):
    def __init__(self):
        super(video_analysis, self).__init__()
        resnet = models.resnet152(pretrained=True)
        in_feats = resnet.fc.in_features
        base_mod = list(resnet.children())[:-1]
        self.base_mod = nn.Sequential(*base_mod)
        for child in self.base_mod.children():
            for param in child.parameters():
                param.requires_grad = False
        self.out1 = nn.Linear(in_feats, 512)
        self.func1 = nn.LeakyReLU(0.1)
        self.bn1 = nn.BatchNorm1d(512)
        self.out2 = nn.Linear(512, 512)
        self.func2 = nn.LeakyReLU(0.1)
        self.bn2 = nn.BatchNorm1d(512)
        self.out3 = nn.Linear(512, 512)
        
        self.LSTM = nn.LSTM(input_size = 512, hidden_size = 256, num_layers = 2, batch_first = True)
        self.fc1 = nn.Linear(256, 128)
        self.func1out = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(128, 32)
    
    def forward(self, x):
        x_embed = []
        for i in range(x.size(1)):
            x_im = self.base_mod(x[:,i,:,:,:])
            x_im = x_im.view(x_im.size(0), -1)
            x_im = self.out1(x_im)
            x_im = self.func1(x_im)
            x_im = self.bn1(x_im)
            x_im = self.out2(x_im)
            x_im = self.func2(x_im)
            x_im = self.bn2(x_im)
            x_im = self.out3(x_im)
            x_embed.append(x_im)
        x_embed = torch.stack(x_embed, dim=0).transpose_(0, 1)
        print(x_embed.shape)
        out_lstm, (ht, states) = self.LSTM(x_embed)
        print('state')
        print(ht.shape)
        out = out_lstm[:, -1, :]
        out = self.fc1(out)
        out = self.func1out(out)
        out = self.fc2(out)
        return out

In [ ]:
class mixed(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers,  glove_weights):
        super().__init__()
        self.text_emb = BiLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, glove_weights)
        self.image = image()
        self.video = video_analysis()
        
        self.categ_1 = nn.Linear(31, 32)
        self.funccateg_1 = nn.LeakyReLU(0.1)
        self.categ_2 = nn.Linear(32, 32)
        
        self.numer_1 = nn.Linear(7, 16)
        self.funcnumer_1 = nn.LeakyReLU(0.1)
        self.numer_2 = nn.Linear(16, 32)
        
        self.fc1 = nn.Linear(160, 128)
        self.func1 = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 32)
        self.func2 = nn.LeakyReLU(0.1)
        self.fc3 = nn.Linear(32, 1)
    def forward(self, text, l, hour, day, numerical, vid, image):
        categ = torch.cat((hour, day),  dim = -1)
        categ = self.categ_1(categ)
        categ = self.funccateg_1(categ)
        categ = self.categ_2(categ)
        
        num = self.numer_1(numerical)
        num = self.funcnumer_1(num)
        num = self.numer_2(num)
        
        text = self.text_emb(text, l)
        vid = self.video(vid)
        img = self.image(image)
        print('text')
        print(text.shape)
        print('num')
        print(num.shape)
        print(vid.shape)
        print(categ.shape)
        print(img.shape)
        multi  = torch.cat([num, categ, text, vid, img], axis = -1)
        
        out = self.fc1(multi)
        out = self.func1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.func2(out)
        out = self.fc3(out)
        return out

In [ ]:
model = nn.DataParallel(mixed(vocab_size, 100, 128, 2, W ).cuda())

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [ ]:
train_log = []
val_log = []
import matplotlib.pyplot as plt
def train_model(model, epoch, lr):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    opt_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5)
    criterion = nn.BCEWithLogitsLoss()
    for i in range(epoch):
        model.train()
        sum_loss = 0.0
        total = 0
        train_log_epoch = []
        train_val_epoch = []
        i = 0
        for elem in train_dl:
            i+=1
            print(i)
            optimizer.zero_grad()
            frames = elem[3].cuda()
            image = elem[4].cuda()
            numbers = elem[0].float().cuda()
            hour = elem[1][:,0:24].float().cuda()
            day = elem[1][:,24:31].float().cuda()
            text = elem[2][0].cuda()
            l = elem[2][1]
            y = elem[5].float().cuda()
            y_pred = model(text, l, hour, day, numbers, frames, image)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            loss = loss.item()
            acc = binary_acc(y_pred, y)
            print(acc)
            train_log_epoch.append(acc)
        train_log.extend(train_log_epoch)
        opt_scheduler.step()
        a = np.mean(val(model, val_dl))
        val_log.append((len(train_log), a))
        plot_history(train_log, val_log, title='loss')
    return model


def val(model, valid_dl):
    model.eval()
    sum_loss = 0.0
    val_epoch = []
    for elem in valid_dl:
        frames = elem[3].cuda()
        image = elem[4].cuda()
        numbers = elem[0].float().cuda()
        hour = elem[1][:,0:24].float().cuda()
        day = elem[1][:,24:31].float().cuda()
        text = elem[2][0].cuda()
        l = elem[2][1]
        y = elem[1].float().cuda()
        y_pred = model(text, l, hour, day, numbers, frames, image)
        acc = binary_acc(y_pred, y)
        val_epoch.append(acc) 
    return val_epoch

In [ ]:
model = train_model(model, 1,0.001)